In [ ]:
!pip install evaluate
!pip install rouge_score

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv("/content/outputBartCnndm.csv")
df1 = pd.read_csv("/content/outputPegasusCnndm.csv")
df2 = pd.read_csv("/content/outputProphetNetCnndm.csv")
df3 = pd.read_csv("/content/outputseasonCnndm.csv")

In [2]:
labelsb = df["Actual Summary"].values
predsb = df["Model Summary"].values
labelsp = df1["Actual Summary"].values
predsp = df1["Model Summary"].values
labelspr = df2["Actual Summary"].values
predspr = df2["Model Summary"].values
labels = df3["Actual Summary"].values
preds = df3["Model Summary"].values


In [4]:
import evaluate
rouge = evaluate.load('rouge')


In [5]:
results = rouge.compute(predictions=predsb,references=labelsb)
print("ROUGE score BART:\n",results)
results = rouge.compute(predictions=predsp,references=labelsp)
print("ROUGE score PEGASUS:\n",results)
results = rouge.compute(predictions=predspr,references=labelspr)
print("ROUGE score ProphetNet:\n",results)
results = rouge.compute(predictions=preds,references=labels)
print("ROUGE score SEASON:\n",results)

ROUGE score BART:
 {'rouge1': 0.33688123173334544, 'rouge2': 0.13472133192582447, 'rougeL': 0.23858245912554832, 'rougeLsum': 0.23863019482504805}
ROUGE score PEGASUS:
 {'rouge1': 0.3410536587258549, 'rouge2': 0.1450344169993145, 'rougeL': 0.25069593621649605, 'rougeLsum': 0.28778047019750164}
ROUGE score ProphetNet:
 {'rouge1': 0.3217772428910745, 'rouge2': 0.12508335630899173, 'rougeL': 0.22602373334815162, 'rougeLsum': 0.2644270859781015}
ROUGE score SEASON:
 {'rouge1': 0.3656460093984095, 'rouge2': 0.16324180859201293, 'rougeL': 0.27175854347338047, 'rougeLsum': 0.2716117199658815}


In [6]:
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.translate import meteor
from nltk import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
meteor_val = []
for i in range(len(preds)):
  same_res = round(meteor([word_tokenize(predsb[i])],word_tokenize(labelsb[i])), 4)
  meteor_val.append(same_res)
print("Meteor score BART:",np.mean(meteor_val))

meteor_val = []
for i in range(len(preds)):
  same_res = round(meteor([word_tokenize(predsp[i])],word_tokenize(labelsp[i])), 4)
  meteor_val.append(same_res)
print("Meteor score PEGASUS:",np.mean(meteor_val))

meteor_val = []
for i in range(len(preds)):
  same_res = round(meteor([word_tokenize(predspr[i])],word_tokenize(labelspr[i])), 4)
  meteor_val.append(same_res)
print("Meteor score ProphetNet:",np.mean(meteor_val))

meteor_val = []
for i in range(len(preds)):
  same_res = round(meteor([word_tokenize(preds[i])],word_tokenize(labels[i])), 4)
  meteor_val.append(same_res)
print("Meteor score SEASON:",np.mean(meteor_val))


Meteor score BART: 0.24859140000000002
Meteor score PEGASUS: 0.294086
Meteor score ProphetNet: 0.22206289999999998
Meteor score SEASON: 0.32721089999999997


In [ ]:
!pip install bert_score
from evaluate import load
bertscore = load("bertscore")

In [9]:
results = bertscore.compute(predictions=predsb, references=labelsb, lang="en")
precision = np.mean(results["precision"])
recall = np.mean(results["recall"])
f1 = np.mean(results["f1"])
print("BERTScore BART->(precision,recall,f1):",precision,recall,f1)

results = bertscore.compute(predictions=predsp, references=labelsp, lang="en")
precision = np.mean(results["precision"])
recall = np.mean(results["recall"])
f1 = np.mean(results["f1"])
print("BERTScore PEGASUS->(precision,recall,f1):",precision,recall,f1)

results = bertscore.compute(predictions=predspr, references=labelspr, lang="en")
precision = np.mean(results["precision"])
recall = np.mean(results["recall"])
f1 = np.mean(results["f1"])
print("BERTScore ProphetNet->(precision,recall,f1):",precision,recall,f1)

results = bertscore.compute(predictions=preds, references=labels, lang="en")
precision = np.mean(results["precision"])
recall = np.mean(results["recall"])
f1 = np.mean(results["f1"])
print("BERTScore SEASON->(precision,recall,f1):",precision,recall,f1)



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore BART->(precision,recall,f1): 0.8718532657623291 0.892842309832573 0.8821229474544525
BERTScore PEGASUS->(precision,recall,f1): 0.8805496315956116 0.8855858129262925 0.8829333598017692
BERTScore ProphetNet->(precision,recall,f1): 0.8589629653692246 0.8828513559103012 0.8706514504551888
BERTScore SEASON->(precision,recall,f1): 0.8887648682594299 0.890691265463829 0.889563225388527


In [ ]:
!pip install moverscore
!pip install pyemd
!pip install pytorch-pretrained-bert

In [3]:
from moverscore import get_idf_dict, word_mover_score
from collections import defaultdict
idf_dict_hyp = get_idf_dict(predsb) # idf_dict_hyp = defaultdict(lambda: 1.)
idf_dict_ref = get_idf_dict(labelsb) # idf_dict_ref = defaultdict(lambda: 1.)
scores = word_mover_score(labelsb, predsb, idf_dict_ref, idf_dict_hyp, \
                          stop_words=[], n_gram=1, remove_subwords=True)
mover_score = np.mean(scores)
print("MoverScore BART",mover_score)

idf_dict_hyp = get_idf_dict(predsp) # idf_dict_hyp = defaultdict(lambda: 1.)
idf_dict_ref = get_idf_dict(labelsp) # idf_dict_ref = defaultdict(lambda: 1.)
scores = word_mover_score(labelsp, predsp, idf_dict_ref, idf_dict_hyp, \
                          stop_words=[], n_gram=1, remove_subwords=True)
mover_score = np.mean(scores)
print("MoverScore PEGASUS",mover_score)

idf_dict_hyp = get_idf_dict(predspr) # idf_dict_hyp = defaultdict(lambda: 1.)
idf_dict_ref = get_idf_dict(labelspr) # idf_dict_ref = defaultdict(lambda: 1.)
scores = word_mover_score(labelspr, predspr, idf_dict_ref, idf_dict_hyp, \
                          stop_words=[], n_gram=1, remove_subwords=True)
mover_score = np.mean(scores)
print("MoverScore ProphetNet",mover_score)

idf_dict_hyp = get_idf_dict(preds) # idf_dict_hyp = defaultdict(lambda: 1.)
idf_dict_ref = get_idf_dict(labels) # idf_dict_ref = defaultdict(lambda: 1.)
scores = word_mover_score(labels, preds, idf_dict_ref, idf_dict_hyp, \
                          stop_words=[], n_gram=1, remove_subwords=True)
mover_score = np.mean(scores)
print("MoverScore SEASON",mover_score)

[--------------------------------------------------]
MoverScore BART 0.13118566788769134
MoverScore PEGASUS 0.1342763746343632
MoverScore ProphetNet 0.11572999061116092
MoverScore SEASON 0.15952389217398616


In [10]:
#average length
total = 0
for i in range(len(preds)):
  total+= len(word_tokenize(labels[i]))
avg = total/len(preds)
print("average length of actual summay:",avg)

total = 0
for i in range(len(preds)):
  total+= len(word_tokenize(predsb[i]))
avg = total/len(preds)
print("average length of BART summay:",avg)

total = 0
for i in range(len(preds)):
  total+= len(word_tokenize(predsp[i]))
avg = total/len(preds)
print("average length of PEGASUS summay:",avg)

total = 0
for i in range(len(preds)):
  total+= len(word_tokenize(predspr[i]))
avg = total/len(preds)
print("average length of ProphetNet summay:",avg)

total = 0
for i in range(len(preds)):
  total+= len(word_tokenize(preds[i]))
avg = total/len(preds)
print("average length of SEASON summay:",avg)



average length of actual summay: 37.282
average length of BART summay: 56.94
average length of PEGASUS summay: 41.918
average length of ProphetNet summay: 59.288
average length of SEASON summay: 38.456
